In [1]:
from matplotlib.colors import LogNorm

In [2]:
# create 2d pink noise from -1.0 to 1.0 for the surface roughness
# pink noise has a natural "cloudy" shape
# make modulated lateral surface around the wire

"""
Created on Fri Jul 31 10:44:22 2015

@author: Melanie Roedel
source: java script: http://www.redblobgames.com/articles/noise/2d/
"""
from scipy import fftpack
def noise2d(dim,exp,display=False,filepath=False,fmin=1,fmax=None):
    """
    noise2d creates a 2d array with noise
    args:
        dim= dimension of array
        exp= exponential of I over frequency I=f^exp
                -1=pink noise
                -2=red/brown noise
                0=white noise
        
    kvargs: 
        filepath = filepath, default=False (no saving)
        fmin==1 minimal frequency
        fmax==None maximal frequency
        will apply a filter in frequency domain and blur the image
    """    
    
    # create random complex numbers
    r=np.random.rand(dim,dim)*dim**2.0
    phi=np.random.rand(dim,dim)*2*np.pi
    random_x=r*np.cos(phi)
    random_y=r*np.sin(phi)

    
    # scale array that falls off with f^exp           
    y,x = np.ogrid[0:dim, 0:dim]
    f = np.sqrt((x-dim//2.0)**2 + (y-dim//2.0)**2)**(2*exp)
    f[dim//2,dim//2]=0
    # fmin and fmax create masks in f-space to blur the image
    if fmin>1:
        innermask=(x-dim//2.0)**2 + (y-dim//2.0)**2 >=fmin**2
        f==f*innermask
    if fmax!=None:
        outermask=(x-dim//2.0)**2 + (y-dim//2.0)**2 <=fmax**2
        f=f*outermask
        
    scale=fftpack.fftshift(f)
    
    # scale random numbers
    real=random_x*scale
    imag=random_y*scale
    
    # ifft real and imag
    rfft=np.fft.ifft2(real+imag*1j)
    realrfft=np.real(rfft)
    rMin = np.amin(realrfft)
    rMax = np.amax(realrfft)
    
    # normalize noise to [0,1]
    noise = (realrfft - rMin) / (rMax - rMin)
    
    if display==True:
        plt.figure(display)
        plt.imshow(noise)
        plt.colorbar()
    
    if filepath!=False:
        print "Saving to",filepath+'pow'+str(exp)+'noise_'\
        +str(dim)+'pix'+'.tif'
        # save noise
        img = Image.fromarray(noise)
        img.save(filepath+'pow'+str(exp)+'noise_'+str(dim)\
                 +'pix'+'.tif')
    return noise


In [3]:
from scipy.special import erf

def theta(x):
    return 0.5 * (np.sign(x) + 1)

import scipy
def logn(x,N):
    return np.log(x)/np.log(N)
def supergaussian(x, A, mu, sigma, offset, N=8):
    """Supergaussian function, amplitude A, centroid mu, st dev sigma, exponent N, with constant offset"""
    denom = np.float64(2*sigma**N)
    return A * (1/(2**(1+1/N)*sigma*2*scipy.special.gamma(1+1/N))) *\
                np.exp(-numpy.absolute(numpy.power(x-mu,N))/denom) *4. *sigma + offset

def inv_supergaussian(y, A, mu, sigma, offset, N=8):
    """Supergaussian function, amplitude A, centroid mu, st dev sigma, exponent N, with constant offset"""
    y[y>A]=0.
    denom = np.float64(2*sigma**N)
    #print (y-offset)/(A * (1/(2**(1+1/N)*sigma*2*scipy.special.gamma(1+1/N))) *4. *sigma)
    #print denom*np.log((y-offset)/(A * (1/(2**(1+1/N)*sigma*2*scipy.special.gamma(1+1/N))) *4. *sigma))
    #print logn(-1.0*denom*np.log((y-offset)/(A * (1/(2**(1+1/N)*sigma*2*scipy.special.gamma(1+1/N))) *4. *sigma)),N)
    return (abs(-1.0*denom*np.log((y-offset)/(A * (1/(2**(1+1/N)*sigma*2*scipy.special.gamma(1+1/N))) *4. *sigma))))**(1./N)+mu

def trapezoid(x,A1,A2,x1,x2):
    m=1.0*(A2-A1)/(x2-x1)
    n=A1-m*x1
    #return (-theta(x-x2)+theta(x-x1))*(m*x+n)
    return (-theta(x-x1)+1)*A1+(-theta(x-x2)+theta(x-x1))*(m*x+n)

# front sharpness:

def sharp_front(x,center,length):
    return theta(x+length/2-center)-theta(x-length/2-center)

def erf_front(x,center,length,width1,width2):
    return (erf((x-(center-length/2.))/width1)-erf((x-(center+length/2.))/width2))/2.

def supergauss_front(x,center,length,sharpness=8):
    return supergaussian(x, 1.0, center, length/2., 0., N=sharpness)

def erf_filled(x,radius,width):
    return (erf(-(x-radius)/width)+1.)/2.

def supergauss_filled(x,radius,sharpness=8):
    return supergaussian(x, 1.0, 0., radius, 0., N=sharpness)

# front curvature:
def square_curvature(y,a=5e-4,y0=None,d=None):
    if y0==None:
        y0=y[len(y)//2]
    if d==None:
        d=y[len(y)//2]
    return -a*(y-y0)**2.0+d

def supergauss_curvature(y,A=None,y0=None,sigma=None,d=None,sharpness=5.):
    if y0==None:
        y0=y[len(y)//2]
    if d==None:
        d=y[len(y)//2]
    if A==None:
        A=y[len(y)//4]
    if sigma==None:
        sigma=y[len(y)//4]
    return supergaussian(y, A, y0, sigma, d, N=sharpness)

par_sharp_filled=[]
par_erf_filled=[10.]
par_supergauss_filled=[10.]
par_erf_front=[75.,10.,30.]
par_sharp_front=[75.]

In [4]:
def cart2polar(x, y,center):
    """
    converts cartesian to polar coordinates
    input:
        x,y: cartesian coordinated
        center: array [x_center,y_center]
    output:
        r,theta: polar coordinates
    """
    r = np.sqrt((x-center[0])**2 + (y-center[1])**2)
    theta = np.arctan2((y-center[1]), (x-center[0]))
    return r, theta

In [5]:
def blurrycircle(imdim, radius,frontparams,center=None,edge='sharp_filled'):
    """
    Creates a uniform filled circle with a blurred edge
    input:
        args:
        imdim: array [xdim,zdim] for the 2d output
        meanradius: average radius of the circle
        kvarg:
        center=None: center of the circle, if center=None it will be set 
                        to imdim//2
    output:
        array with noisy uniformly filled circle and blurred edge
    """
    # define center to center of image    
    if center==None:
        center=np.array([imdim[0]//2,imdim[1]//2])
    # create 2D circle
    zz,xx=np.meshgrid(np.arange(imdim[1]),np.arange(imdim[0])) 
    
    r,phi=cart2polar(xx,zz,center)
    if edge=='sharp_filled':
        circ=r<radius
        circ=circ*1.0
    if edge=='sharp_front':
        circ=sharp_front(r,radius,*frontparams)
    if edge=='erf_front':
        circ=erf_front(r,radius,*frontparams)
    if edge=='supergauss_front':
        circ=supergauss_front(r,radius,*frontparams)
    if edge=='erf_filled':
        circ=erf_filled(r,radius,*frontparams)
    if edge=='supergauss_filled':
        circ=supergauss_filled(r,radius,*frontparams)
    return circ

#plt.imshow(blurrycircle([512,512],512/4.,edge='sharp_front',frontparams=par_sharp_front))
#plt.colorbar()

In [6]:
def heatedwire(xdim,ydim,zdim,sigma,\
                  center=None,\
                  display=True,status=True,save=False,savepath='./',\
                  curvature='supergauss',curve_param=None,edge_mode='sharp_filled',
                  smooth_mode='full',noise=0.,noisearray=[],
                  filename='heatedwiremodel'):
    """
    Creates a bent cylinder with uniform density and a noisy edge and returns 
    its projection along a radius or the 3D array
    input:
        args:
        xdim,ydim,zdim: output dimensions of the array
        front_params:parameters for edge smoothness, for 'sharp_filled': []
        kvarg:
        center=None: center of the circle, if center=None it will be set 
                        to imdim//2
        display=True: plot created projection and/or slices of the cylinder
        status=True: print status of calculation (number of calculated slices)
        savepath='./': path, where the prjection and/or slices will be saved
        curvature='supergauss': function that if rotated results in the body of revolution
        curve_param=None: Parameters that are needed for the curvature-function, 
                                if none, standard parameters are used
                                in case of 'supergauss': [thickness,center,width,N,radius]
        edge_mode='sharp_filled': Is the edge sharp or blurry?
    output:
        proj: 2D projection along a radius
        
    """
    import os
    # create savepath
    if not os.path.exists(savepath):
        os.makedirs(savepath)
        print "Created", savepath
    
    # initialize arrays for cylinder/projection
    if center==None:
        center=[xdim//2,zdim//2]
    xc=center[0]
    zc=center[1]
    
    proj=np.zeros((ydim,xdim))
    
    zz,xx=np.meshgrid(np.arange(xdim),np.arange(ydim)) 
    r,phi=cart2polar(xx,zz,[size//2,size//2])
    phiind=np.around((phi+np.pi)/(2.*np.pi/(xdim-1)))
    phiind=phiind.astype(int)
    
    
    if curve_param==None:
        print 'thickness,center,width,N,radius,erfwidth',[ydim/10.,ydim/2.,ydim/6.,5,ydim/3.,ydim/20.]
        thickness,center,width,N,radius=[ydim/10.,ydim/2.,ydim/6.,5,ydim/3.,ydim/20.]
    else:
        thickness,center,width,N,radius,erfwidth=curve_param
    if curvature=='supergauss':
        r=-supergaussian(np.arange(ydim),thickness,center,width,0.,N)+radius
        #-supergaussian(np.arange(ydim),ydim//10,ydim//2,ydim/6.,0.,5)+ydim//3
    elif curvature=='erf':
        r=-thickness*(erf((np.arange(ydim)-(center-width))/erfwidth)\
                           -erf((np.arange(ydim)-(center+width))/erfwidth))/2.+radius
    elif curvature=='none':
        r=np.ones(ydim)*radius
#    if noise!=0.:
#        r=r*(1.0+noise*noisearray)
    
    if smooth_mode=='full':
        sigmalist=np.ones(ydim)*sigma
    elif smooth_mode=='supergauss':
        sigmalist=supergaussian(np.arange(ydim),sigma,center,width,0.,N)
    elif smooth_mode=='erf':
        sigmalist=sigma*(erf((np.arange(ydim)-(center-width))/erfwidth)-erf((np.arange(ydim)-(center+width))/erfwidth))/2.
    
    for k in range(ydim):
        if status==True:
            #vprint status of slice calculation
            if k % 50==0:
                print "%03d" %k,'/',ydim
        radius2d=np.ones(zdim)*r[k]*(1.+noise*noisearray[k,:])
        wslice=blurrycircle([xdim,zdim],radius2d[phiind],edge=edge_mode,frontparams=[sigmalist[k]], center=[xc,zc])
        proj[k,:]=wslice.sum(axis=1)
    proj[np.isnan(proj)]=0.    
            
    # project the 2D circular slice along z direction and save this
    # 1D projection to the according y position of the projection array
    if display==True:
        plt.figure()
        plt.imshow(proj)
        plt.colorbar()
    if save==True:
        im_proj = Image.fromarray(proj)
        im_proj.save(savepath+filename+'_projection_sum.tif')
        print 'Saved image to ', str(savepath+filename+'_projection_sum.tif')
    return proj

In [7]:
def make_target(target,pixsize,Z,rho,molarmass):
    """
    make_target converts an array (image) in relative units of mass density to
    the number of electrons per pixel
    args: 
        target= array of target density in units of the uncompressed density 
                at room temperature (value 1.0 equals one uncompressed density)
        pixsize= pixel size of one pixel of the given array in m
        thickness = thickness of target over which the density is projected in m
        Z= atomic number of target material
        rho = mass density of target material in g/cm^3
        molarmass = molar mass of target material in g/mol
    return:
        array of number density that has the same dimensions as the original array
    """
    # number of electrons per cm³: avogadro constant * Z * rho / molarmass
    # number of electrons per voxel= avogadro constant * Z * rho / molarmass*voexelvolume (in cm^3)
    
    N=target*6.022e23*Z*rho/molarmass*pixsize**3.0*1.0e6
    N_tot=N.sum()
    print "Total number of electrons: ",'%.3e' % N_tot
    print "Electron number in pixel [",len(target)//2,",",len(target)//2,"]: ", '%.3e' % N[len(target)//2,len(target)//2]
    return N

In [8]:
from scipy import fftpack

def propagate(target, photonnumber, sim_pixsize, dist_det,trans,eff, wavelength,fftsampling=1):
    """
    propagate propagates a given plane wave with the intensity distribution
    illu through a target with a given distribution of electrons to a detector
    plane in the far field by simple 2D FFT
    args:
        target= array with number of electrons in each pixel
        photonnumber= total number of photons I_0
        sim_pixsize= pixelsize of the target array in m
        dist_det= distance target - detector plane
        trans= transmission through the target (absorbed photons will simply 
                    be subtracted by number of incoming photons)
        eff = efficiency of detection
        wavelength = wavelength of x-rays in m
    return:
        I= detected intensity in the detector plane
        diffr_pix = pixelsize of I in the detector plane
        norm = factor that is multiplied with the FFT
    """
    theta_min=np.arcsin(wavelength/(2.*len(target)*sim_pixsize))
    # field of view in the detector plane = size of one pixel in the 
    #   target plane
    # diffr_pixsize = d_det* tan(2*theta_min)
    diffr_pixsize=dist_det*np.tan(2.0*theta_min)
    #print "sum of target", target.sum()
    print "Pixelsize of simulated diffraction pattern", '%.2e' % diffr_pixsize, " m"
    
    # propagate to the Fraunhofer regime by simple FFT
    F1=fftpack.fft2(target,shape=(target.shape*np.ones(2)*fftsampling))
    # shift intensity from the corners to the center (standard FFT procedure)
    F2 = fftpack.fftshift( F1 )/len(target)
    # calculate the minimal scattering 2theta_min
    # theta_min = asin(lambda/(2*biggest feature size))
    # (biggest feature size= size of the simulated target = number of all
    #   pixels in the array* pixelsize -> smallest q =one pixel in diffraction
    #   pattern)
    
    #print "Maximum of the Fourier transform 2: ", '%.2e' %np.amax(abs(F2))
    #print "Maximum of the Fourier transform 1: ", '%.2e' %np.amax(abs(F1))
    
    # calculate prefactor of FFT in correct units
    # r_0^2=classical electron radius^2=7.9408e-30m^2
    norm= photonnumber/sim_pixsize**2.0*diffr_pixsize**2.0/dist_det**2.0*trans*eff*7.9408e-30
    #print "Normalization factor: ", norm
    # intensity = |amplitude of wavefield|^2
    I=norm*np.abs(F2)**2.0
    
    maskcenter=np.ones(I.shape)
    maskcenter[len(maskcenter)//2-3:len(maskcenter)//2+3,len(maskcenter)//2-3:len(maskcenter)//2+3]=0.

    print "Number of scattered photons :", '%.2e' %(I*maskcenter).sum()
    print "Percentage of scattered photons:", '%.2f' %((I*maskcenter).sum()/photonnumber*100.) , "percent"

    return I

In [9]:
#Parameters in pixel

size=4096
radius=1.*size/4./2.
thickness=radius/5.
center=size//2
width=size/4.
N=4
erfwidth=size/9.

shape=[thickness,center,width,N,radius,erfwidth]

sigma_smooth=size/200./2.

print "Parameters in pixel:"
print "Image size", size
print "Radius of full wire", radius
print "Thickness of indent in wire", thickness
print "Radius of indenting optical laser", width
print "Sigma of error function of curvature", erfwidth
print "Edge sharpness of heated area", sigma_smooth

Parameters in pixel:
Image size 4096
Radius of full wire 512.0
Thickness of indent in wire 102.4
Radius of indenting optical laser 1024.0
Sigma of error function of curvature 455.111111111
Edge sharpness of heated area 10.24


In [10]:
#Parameters in SI units
wire_radius_in_m=5e-6/2.
foilpixsize=wire_radius_in_m/radius

print "Parameters in SI units:"
print "Full field of view in microns:", foilpixsize*size*1e6
print "Radius of full wire in micron: " , wire_radius_in_m*1e6
print "Pixelsize in nm:", foilpixsize*1e9
print "Thickness of indent in wire in micron", thickness*foilpixsize*1e6
print "Radius of indenting optical laser in micron", width*foilpixsize*1e6
print "Sigma of error function of curvature in micron", erfwidth*foilpixsize*1e6
print "Edge sharpness of heated area in nm", sigma_smooth*foilpixsize*1e9

#Aluminium:
Z=13
rho=2.70
M=26.9815

print "Number of electrons in 1cm^3:",'%.3e' %(6.022e23*Z*rho/M)
print "Volume of one voxel:",'%.3e' %(foilpixsize**3.), 'm^3'
print "Number of electrons in one voxel if completely filled:",'%.3e' %(6.022e23*Z*rho/M*foilpixsize**3.0*1.0e6)

Parameters in SI units:
Full field of view in microns: 20.0
Radius of full wire in micron:  2.5
Pixelsize in nm: 4.8828125
Thickness of indent in wire in micron 0.5
Radius of indenting optical laser in micron 5.0
Sigma of error function of curvature in micron 2.22222222222
Edge sharpness of heated area in nm 50.0
Number of electrons in 1cm^3: 7.834e+23
Volume of one voxel: 1.164e-25 m^3
Number of electrons in one voxel if completely filled: 9.120e+04


In [11]:
#Other parameters
noise=0.02 # noiselevel (e.g. 5%)

N_xray=1e12
ddet=4.15
#camera_pixsize=13.5e-6
energy=5500.
wavelength=6.626e-34*2.998e8/(energy*1.602e-19)
print "Wavelength: ",wavelength

Wavelength:  2.25453955283e-10


In [12]:
print "Size of target and diffraction pattern in pixel", size
print "Size of target in m", foilpixsize*size
qpixsize=2.*np.pi/(foilpixsize*size)
print "Pixelsize in reciprocal space in 1/m", qpixsize
#q=np.arange(len(trapezoid_broadlines[0]))*qpixsize

NameError: name 'np' is not defined

Size of target and diffraction pattern in pixel 4096
Size of target in m 2e-05


In [13]:
print 2.*np.pi/(foilpixsize*590.)

NameError: name 'np' is not defined

In [14]:
import os

print "Generating 2d pink noise"
noise1=noise2d(size,-1.,display=False)*2.0-1.0
rand = int(os.urandom(4).encode('hex'), 16)
np.random.seed(rand)
noise07=noise2d(size,-0.7,display=False)*2.0-1.0
rand = int(os.urandom(2).encode('hex'), 16)
np.random.seed(rand)
noise08=noise2d(size,-0.8,display=False)*2.0-1.0

noisearr2d=noise1+noise07+noise08 # exp=-1 for pink noise

noise1d=noisearr2d[:,size//2]
print "Mean of pink noise: ", np.mean(noisearr2d),"+/-",np.std(noisearr2d)


NameError: global name 'np' is not defined

Generating 2d pink noise


In [15]:
plt.figure()
plt.plot(noise1d)
plt.figure()
plt.imshow(noisearr2d)
plt.colorbar()
im_targ = Image.fromarray(noisearr2d)
#im_targ.save('./heatedwire/noise_test3.tif')

NameError: name 'plt' is not defined

In [16]:
#test shape:



sgauss=-supergaussian(np.arange(size),thickness,center,width,0.,N)+radius
errorf=-thickness*(erf((np.arange(size)-(center-width))/erfwidth)-erf((np.arange(size)-(center+width))/erfwidth))/2.+radius
plt.plot(np.arange(size),sgauss,label='supergauss curvature')
plt.plot(np.arange(size),errorf,label='error function curvature')
plt.plot(np.arange(size),errorf*(1.+noise*noise1d),label='error function curvature with noise')
plt.ylim([0,size//2])
plt.ylabel('radius')
plt.legend()

NameError: name 'np' is not defined

In [17]:
#test smoothness sigma:
plt.plot(np.arange(size),supergaussian(np.arange(size),sigma_smooth,center,width,0.,N))
plt.plot(np.arange(size),sigma_smooth*(erf((np.arange(size)-(center-width))/erfwidth)-erf((np.arange(size)-(center+width))/erfwidth))/2.)

NameError: name 'plt' is not defined

In [18]:
print foilpixsize

4.8828125e-09


In [19]:
#2D array as a simple example
#errorf=-thickness*(erf((np.arange(size)-(center-width))/erfwidth)-erf((np.arange(size)-(center+width))/erfwidth))/2.+radius
xx,yy=np.meshgrid(np.arange(size),np.arange(size))
proj2d_erf=(yy>(-errorf*(1.)+center))
proj2d_erf=proj2d_erf*\
                (yy<(errorf*(1.)+center))
proj2d_erf=np.rot90(proj2d_erf)*1.0*radius
target2d_erf=make_target(proj2d_erf,foilpixsize,Z,rho,M)

diffr2d_erf=propagate(target2d_erf, N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)
plt.figure()
plt.imshow(target2d_erf)
plt.colorbar()
plt.figure()
plt.imshow(diffr2d_erf, norm=LogNorm(vmin=1, vmax=1e12))
plt.colorbar()
im_targ = Image.fromarray(proj2d_erf)
im_targ.save('./heatedwire/'+str(size)+'/flat_target_no_noise.tif')
im_targ = Image.fromarray(diffr2d_erf)
im_targ.save('./heatedwire/'+str(size)+'/flat_diffr_no_noise.tif')

NameError: name 'np' is not defined

In [20]:
#2D array as a simple example
#errorf=-thickness*(erf((np.arange(size)-(center-width))/erfwidth)-erf((np.arange(size)-(center+width))/erfwidth))/2.+radius
xx,yy=np.meshgrid(np.arange(size),np.arange(size))
proj2d_erf=(yy>(-errorf*(1.+noise*noise1d)+center))
proj2d_erf=proj2d_erf*\
                (yy<(errorf*(1.+noise*noise1d)+center))
proj2d_erf=np.rot90(proj2d_erf)*1.0*radius
target2d_erf=make_target(proj2d_erf,foilpixsize,Z,rho,M)

diffr2d_erf=propagate(target2d_erf, N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)
plt.figure()
plt.imshow(target2d_erf)
plt.colorbar()
plt.figure()
plt.imshow(diffr2d_erf, norm=LogNorm(vmin=1, vmax=1e12))
plt.colorbar()
im_targ = Image.fromarray(proj2d_erf)
im_targ.save('./heatedwire/'+str(size)+'/flat_target.tif')
im_targ = Image.fromarray(diffr2d_erf)
im_targ.save('./heatedwire/'+str(size)+'/flat_diffr.tif')

NameError: name 'np' is not defined

In [21]:
test=heatedwire(size,size,size,sigma_smooth,\
                  center=None,\
                  display=False,status=True,\
                  curvature='none',curve_param=shape,edge_mode='sharp_filled',smooth_mode='full',noise=noise,noisearray=noisearr2d,save=False)
test=make_target(test,foilpixsize,Z,rho,M)
#im_targ = Image.fromarray(supergauss1d)
#im_targ.save('./3dshock/supergauss/'+'electron_density_'+str(ydim)+'pixel_'+'sharp_2d'+'.tif')
diffrtest=propagate(test, N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)
plt.figure()
plt.imshow(test)
im_targ = Image.fromarray(test)
im_targ.save('./heatedwire/'+str(size)+'/test_target_straight.tif')
im_targ = Image.fromarray(diffrtest)
im_targ.save('./heatedwire/'+str(size)+'/test_diffr_straight.tif')

test2=heatedwire(size,size,size,sigma_smooth,\
                  center=None,\
                  display=False,status=True,\
                  curvature='erf',curve_param=shape,edge_mode='erf_filled',smooth_mode='erf',noise=noise,noisearray=noisearr2d,save=False)
test2=make_target(test2,foilpixsize,Z,rho,M)
#im_targ = Image.fromarray(supergauss1d)
#im_targ.save('./3dshock/supergauss/'+'electron_density_'+str(ydim)+'pixel_'+'sharp_2d'+'.tif')
diffrtest2=propagate(test2, N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)
plt.figure()
plt.imshow(test2)
im_targ = Image.fromarray(test2)
im_targ.save('./heatedwire/'+str(size)+'/test_target_curved.tif')
im_targ = Image.fromarray(diffrtest2)
im_targ.save('./heatedwire/'+str(size)+'/test_diffr_curved.tif')
#plt.colorbar()
#plt.figure()
#plt.imshow(diffrtest2, norm=LogNorm(vmin=1, vmax=1e12))

NameError: name 'noisearr2d' is not defined

In [22]:
print sigma_smooth

10.24


In [23]:
test3=heatedwire(size,size,size,sigma_smooth,\
                  center=None,\
                  display=False,status=True,\
                  curvature='erf',curve_param=shape,edge_mode='sharp_filled',smooth_mode='erf',noise=noise,noisearray=noisearr2d,save=False)
test3=make_target(test3,foilpixsize,Z,rho,M)
#im_targ = Image.fromarray(supergauss1d)
#im_targ.save('./3dshock/supergauss/'+'electron_density_'+str(ydim)+'pixel_'+'sharp_2d'+'.tif')
diffrtest3=propagate(test3, N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)
plt.figure()
plt.imshow(test3)
im_targ = Image.fromarray(test3)
im_targ.save('./heatedwire/'+str(size)+'/test_target_curved_sharp.tif')
im_targ = Image.fromarray(diffrtest3)
im_targ.save('./heatedwire/'+str(size)+'/test_diffr_curved_sharp.tif')

NameError: name 'noisearr2d' is not defined

In [24]:
plt.imshow(diffrtest3,vmin=0,vmax=12.)
plt.colorbar()

NameError: name 'plt' is not defined

In [25]:
print "shape=[thickness,center,width,N,radius,erfwidth]"
print shape
print "sigma_smooth"
print sigma_smooth

for modify_noise in np.linspace(0.01,0.05,5):
    print "Sharp", ", Noise: ", str(int(modify_noise*100.))
    proj=heatedwire(size,size,size,sigma_smooth,\
                  center=None,\
                  display=False,status=False,\
                  curvature='erf',curve_param=shape,edge_mode='sharp_filled',smooth_mode='erf',noise=modify_noise,noisearray=noisearr2d,save=False)
    target=make_target(proj,foilpixsize,Z,rho,M)
    im_targ = Image.fromarray(target)
    im_targ.save('./heatedwire/'+str(size)+'/'+'electron_density_'+"sharp_"+str(int(modify_noise*100.))+'percent'+'.tif')
    diffr=propagate(target, N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)
    im_targ = Image.fromarray(diffr)
    im_targ.save('./heatedwire/'+str(size)+'/'+'diffr_'+"sharp_"+str(int(modify_noise*100.))+'percent'+'.tif')
        

for modify_sigma in np.linspace(5.,50.,10.):
    for modify_noise in np.linspace(0.01,0.05,5):
        print "Sigma: ",str(int(modify_sigma)), ", Noise: ", str(int(modify_noise*100.))
        proj=heatedwire(size,size,size,modify_sigma,\
                  center=None,\
                  display=False,status=False,\
                  curvature='erf',curve_param=shape,edge_mode='erf_filled',smooth_mode='erf',noise=modify_noise,noisearray=noisearr2d,save=False)
        target=make_target(proj,foilpixsize,Z,rho,M)
        im_targ = Image.fromarray(target)
        im_targ.save('./heatedwire/'+str(size)+'/'+'electron_density_'+"_sigma"+str(int(modify_sigma))+"px_noise"+str(int(modify_noise*100.))+'percent'+'.tif')
        diffr=propagate(target, N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)
        im_targ = Image.fromarray(diffr)
        im_targ.save('./heatedwire/'+str(size)+'/'+'diffr_'+"_sigma"+str(int(modify_sigma))+"px_noise"+str(int(modify_noise*100.))+'percent'+'.tif')
print "Done"

NameError: name 'np' is not defined

shape=[thickness,center,width,N,radius,erfwidth]
[102.4, 2048, 1024.0, 4, 512.0, 455.1111111111111]
sigma_smooth
10.24


In [26]:
im_targ = Image.fromarray(test2)
im_targ.save('./heatedwire/target_test2_2048.tif')
im_targ = Image.fromarray(diffrtest2)
im_targ.save('./heatedwire/diffr_test2_2048.tif')
plt.imshow(diffrtest2, norm=LogNorm(vmin=1, vmax=1e12))
plt.colorbar()

NameError: name 'Image' is not defined

In [27]:
# array for modes
#[curvature,edge_mode,smooth_mode]
par_array=\
[
 ['supergauss','sharp_filled','full'],
 ['supergauss','erf_filled','full'],
 ['supergauss','erf_filled','supergauss'],
 ['erf','sharp_filled','full'],
 ['erf','erf_filled','full'],
 ['erf','erf_filled','erf']
 ]
print "Number of modes:", len(par_array)

Number of modes: 6


In [28]:
projections=[]

for m in range(len(par_array)):
    print "Parameters:", par_array[m]
    single_proj=heatedwire(size,size,size,sigma_smooth,\
                      center=None,\
                      display=False,status=False,savepath='./heatedwire/0.03noise/',\
                      curvature=par_array[m][0],curve_param=shape,edge_mode=par_array[m][1],save=False,\
                      smooth_mode=par_array[m][2],noise=0.03,noisearray=noise2d)
    projections.append(single_proj)
print "Done"

NameError: global name 'np' is not defined

Parameters: ['supergauss', 'sharp_filled', 'full']
Created ./heatedwire/0.03noise/


In [29]:
fig, axs = plt.subplots(3,2)

axs = axs.ravel()

for m in range(len(par_array)):
    cax=axs[m].imshow(projections[m])
    axs[m].set_title(par_array[m][0]+'_'+par_array[m][1]+'_'+par_array[m][2],fontsize=10)
    axs[m].set_axis_off()
    
plt.tight_layout()

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(cax, cax=cbar_ax)
#plt.savefig('./heatedwire/supergauss_'+str(size)+'pixel_'+'summary'+'_projection_sum.jpg')

NameError: name 'plt' is not defined

In [30]:
plt.imshow(projections[0]-projections[-1])

NameError: name 'plt' is not defined

In [31]:


"""
size=512

thickness=50.
center=256.
width=80.
N=5
radius=170.
shape=[thickness,center,width,N,radius]
sigma_smooth=20.
"""



targets=[]
for m in range(len(par_array)):
    #print modes[m]
    single_target =make_target(projections[m],foilpixsize,Z,rho,M)
    #im_ed = Image.fromarray(single_target)
    #im_ed.save('./heatedwire/supergauss_electron_density_'+str(size)+'pixel_'+titles[m]+'.tif')
    targets.append(single_target)
print "Done"

IndexError: list index out of range

In [32]:
fig, axs = plt.subplots(3,2)

axs = axs.ravel()

for m in range(len(par_array)):
    cax=axs[m].imshow(targets[m])
    axs[m].set_title(par_array[m][0]+'_'+par_array[m][1]+'_'+par_array[m][2],fontsize=10)
    axs[m].set_axis_off()
    
plt.tight_layout()

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(cax, cax=cbar_ax)

NameError: name 'plt' is not defined

In [33]:







maskcenter=np.ones([size,size])
maskcenter[len(maskcenter)//2-3:len(maskcenter)//2+3,len(maskcenter)//2-3:len(maskcenter)//2+3]=0.


diffpatterns=[]
for m in range(len(projections)):
    print par_array[m][0]+'_'+par_array[m][1]+'_'+par_array[m][2]
    diffr=propagate(targets[m], N_xray, foilpixsize, dist_det=ddet,trans=1.0,eff=1.0,wavelength=wavelength)

    print "Mask out central 9 pixels"
    print "Number of scattered photons :", '%.2e' %(diffr*maskcenter).sum()
    print "Percentage of scattered photons:", '%.2f' %((diffr*maskcenter).sum()/N_xray*100) , "percent"

    #print "Mask out central pixels"
    #print "central pixel", diffr[len(diffr)//2,len(diffr)//2]
    #print "Number of scattered photons :", '%.2e' %((diffr).sum()-diffr[len(diffr)//2,len(diffr)//2])
    #print "Percentage of scattered photons:", '%.2f' %(((diffr).sum()-diffr[len(diffr)//2,len(diffr)//2])/N_xray*100) , "percent" 

    diffpatterns.append(diffr)

print "Done"

NameError: name 'np' is not defined

In [34]:
fig, axs = plt.subplots(3,2)

axs = axs.ravel()

for m in range(len(projections)):
    cax=axs[m].imshow(diffpatterns[m], norm=LogNorm(vmin=1, vmax=1e12))
    axs[m].set_title(par_array[m][0]+'_'+par_array[m][1]+'_'+par_array[m][2],fontsize=10)
    axs[m].set_axis_off()
    #im_diffr = Image.fromarray(diffpatterns_supergauss[m])
    #im_diffr.save('./heatedwire/supergauss_'+'diffpattern_'+str(size)+'pixel_'+titles[m]+'.tif')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(cax, cax=cbar_ax)


NameError: name 'plt' is not defined

In [35]:
#Evaluate whiskers

In [36]:
from scipy import ndimage

def linescan(i,inds,rstep=None,interp=None,spline_order=1,plot=None,ax=None,label='line profile',color='red',frame=None):
    y0,y1,x0,x1=inds
    #print inds
    if frame==None:
        frame,fx,fy=frame_image(i)
        x0+=fx
        x1+=fx
        y0+=fy
        y1+=fy
    elif frame=='done':
        frame=i
        
    if rstep==None:
        rstep=int(np.sqrt((y1-y0)**2+(x1-x0)**2))
    x, y = np.linspace(x0, x1, rstep), np.linspace(y0, y1, rstep)
    
    if interp==None:
        li = frame[x.astype(np.int), y.astype(np.int)]
    else:
        li = ndimage.map_coordinates(i, np.vstack((x,y)), order=spline_order, mode=interp)    
    if plot=='single':
        fig, axes = plt.subplots(nrows=2)
        axes[0].imshow(frame)
        axes[0].plot([y0, y1], [x0, x1], 'ro-') 
        axes[1].plot(li,color=color)
    if plot=='multi':
        axes=ax
        axes[1].plot(li,label=label,color=color) 
        axes[0].plot([y0, y1], [x0, x1], linestyle='-',color=color)
        axes[0].plot([y0],[x0], 'ro',color='blue')
    return li

def gaussian2(x, a1,b1,c1, a2,b2,c2, d):
   val = a1*np.exp(-(x - b1)**2 / (2*c1**2)) + a2*np.exp(-(x - b2)**2 / (2*c2**2)) + d
   return val

def gaussian3(x, a1,b1,c1, a2,b2,c2, a3,b3,c3, d):
   val = a1*np.exp(-(x - b1)**2 / (2*c1**2)) + a2*np.exp(-(x - b2)**2 / (2*c2**2)) + a3*np.exp(-(x - b3)**2 / (2*c3**2)) + d
   return val

def gaussian4(x, a1,b1,c1, a2,b2,c2, a3,b3,c3, a4,b4,c4, d):
   val = a1*np.exp(-(x - b1)**2 / (2*c1**2)) + a2*np.exp(-(x - b2)**2 / (2*c2**2)) + a3*np.exp(-(x - b3)**2 / (2*c3**2)) + a4*np.exp(-(x - b4)**2 / (2*c4**2)) + d
   return val 

def linear(x,a,b):
    val=a+b*x
    return val

In [37]:
from skimage.transform import rotate
def broad_linescan(image,angle,center,framelength,framewidth,backgrounderr=None,plot=False,log=False,mode='mean'):
    rot=rotate(image, angle, resize=False, center=center, preserve_range=True)
    framelength=int(framelength)
    if plot==True:
        plt.figure()
        ax11=plt.subplot(121)
        plt.imshow(image,vmin=0, vmax=np.amax(image))
        ax11.set_xticks([0,1000,2000])
        ax11.set_yticks([0,1000,2000])
        if log==True:
            plt.imshow(image, norm=LogNorm(vmin=1, vmax=np.amax(image)))
        plt.colorbar(shrink=0.7)
        plt.scatter(center[0],center[1],color='deeppink')
        plt.plot([center[0],center[0]+framelength*np.cos(angle*2.*np.pi/360.)],[center[1],\
        center[1]+framelength*np.sin(angle*2.*np.pi/360.)],\
        linewidth=1,label='Lineprofile',color='gold')
        ax12=plt.subplot(122)
        plt.imshow(rot,vmin=0, vmax=np.amax(image))
        gca().add_patch(Rectangle((center[0],center[1]-framewidth//2), framelength, framewidth, facecolor='none', edgecolor='green'))
        #plt.axvspan(center[1]-framewidth//2, center[1]+framewidth//2, \
        #ymin=center[0], ymax=center[0]+framelength, facecolor='g',alpha=0.5,zorder=10)
        ax12.set_xticks([0,1000,2000])
        ax12.set_yticks([0,1000,2000])
        if log==True:
            plt.imshow(rot, norm=LogNorm(vmin=1, vmax=np.amax(image)))
        plt.colorbar(shrink=0.7)
        plt.scatter(center[0], center[1],color='deeppink')
        plt.tight_layout()

    #print "shape rot",rot.shape
    streakframe=rot[center[1]-framewidth//2:center[1]+framewidth//2,center[0]:center[0]+framelength]
    if streakframe.shape[1]!=framelength:
        print "Framelength:",framelength
        print "Streakframe:",len(streakframe)
        streakframe=rot[center[1]-framewidth//2:center[1]+framewidth//2,center[0]:center[0]+framelength+1]
        print "Streakframe:",len(streakframe)
    
    if backgrounderr!=None:
        roterr=rotate(backgrounderr, angle, resize=False, center=center, preserve_range=True)
        errframe=roterr[center[1]-framewidth//2:center[1]+framewidth//2,center[0]:center[0]+framelength]
        if streakframe.shape[1]!=framelength:
            errframe=roterr[center[1]-framewidth//2:center[1]+framewidth//2,center[0]:center[0]+framelength+1]
    #center[0]:center[0]+framelength
    #print "Framelength", framelength
    #print "coordinates to cut out",center[0],center[0]+framelength
    print "Average lineprofile over rectangle of shape", streakframe.shape
    if mode=='mean':
        meanline=np.mean(streakframe,axis=0)
        if backgrounderr!=None:
            bgerr=np.sqrt(np.sum(errframe**2.,axis=0))/np.sqrt(framewidth)
            err=np.sqrt(meanline+bgerr**2.)
    elif mode=='sum':
        meanline=np.sum(streakframe,axis=0)
        if backgrounderr!=None:
            bgerr=np.sqrt(np.sum(errframe**2.,axis=0))
            print "Bgerr:", np.mean(bgerr)
            err=np.sqrt(meanline+bgerr**2.)
            print "Poisson err:", np.mean(np.sqrt(abs(meanline)))
            #print "Error total", np.mean(err)
    if plot=='section':
        fig=plt.figure()
        ax3 = fig.add_subplot(111)
        sec=ax3.imshow(streakframe,vmin=0, vmax=np.amax(image))
        ax3.spines['top'].set_color('green')
        ax3.spines['bottom'].set_color('green')
        ax3.spines['left'].set_color('green')
        ax3.spines['right'].set_color('green')
        ax3.set_xticklabels([])
        ax3.set_yticklabels([])
        if log==True:
            sec=ax3.imshow(streakframe, norm=LogNorm(vmin=1, vmax=1e14))
        #fig.colorbar(sec)
    if plot==True:
        fig=plt.figure()
        ax3 = fig.add_subplot(111)
        im_frame=ax3.imshow(streakframe,vmin=0, vmax=np.amax(image))
        ax3.spines['top'].set_color('green')
        ax3.spines['bottom'].set_color('green')
        ax3.spines['left'].set_color('green')
        ax3.spines['right'].set_color('green')
        ax3.set_xticklabels([])
        ax3.set_yticklabels([])
        if log==True:
            im_frame=ax3.imshow(streakframe, norm=LogNorm(vmin=1, vmax=1e8))
        fig.colorbar(im_frame)
        fig2=plt.figure()
        ax4 = fig2.add_subplot(111)
        if backgrounderr!=None:
            #plt.errorbar(np.arange(framelength), meanline, yerr=err,color='black')
            plt.fill_between(np.arange(framelength), meanline+err,meanline-err,linewidth=0,color='b')
            plt.fill_between(np.arange(framelength), meanline+np.sqrt(meanline),meanline-np.sqrt(meanline),linewidth=0,color='g')
        #plt.ylim([0.0,200.0])
        
        plt.plot(np.arange(len(rot[center[1],center[0]:center[0]+framelength+1])),rot[center[1],center[0]:center[0]+framelength+1]*framewidth,\
                 color='black',label='1 pixel linescan multiplied by '+\
                 str(int(framewidth)))
        plt.plot(np.arange(framelength),meanline,color='green',label=str(int(framewidth))+' pixel linescan')
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        plt.ylabel('I in ADU')
        plt.xlabel('q in 1/m')
        plt.xscale('log')
        plt.yscale('log')
        plt.legend(loc=3)
        
    if backgrounderr==None:
        return meanline
    else:
        return meanline,err

ImportError: No module named skimage.transform

In [38]:
#-------trapezoid find whisker

#------------make vertical linescans-----------------
#%matplotlib qt

im=trapezoiddiffr1d
im[:,0:len(im)//2]=0.

fig, axes = plt.subplots(nrows=2)
axes[0].imshow(log(im))

gauss4p0=[2500.,1032.,8.,400.,1018.,40.,400.,895.,15.,200.,1150.,30.,20.]
gauss2p0=[1500.,900.,20.,1500.,1050.,30.,0.]
gauss3p0=[3e10,210.,5.,7e10,250.,5.,3e10,290.,5.,0.]
x=np.linspace(len(im)//2+10,310,30)
yaxis=np.arange(len(im))
gausspararray=np.ones((len(gauss3p0),len(x)))*np.nan
gausserrarray=np.ones((len(gauss3p0),len(x)))*np.nan

fitrange=np.array([10.,500.])


for i in np.arange(len(x)):
#for i in np.arange(2):

    #print x[i]
    line=linescan(im,[x[i],x[i],0,len(im)],spline_order=5,plot='multi',interp='nearest',ax=axes,color='blue',label=str(x[i]),frame='done')
    #file=open(path+'testverticalline.txt','w+')
    #for l in range(len(line)):
    #     file.write(str(l)+'\t'+str(line[l])+'\n')
    #file.close()
    try:
        if i==0:
            gauss4par,gauss4cov=curve_fit(gaussian3,yaxis[fitrange[0]:fitrange[1]],line[fitrange[0]:fitrange[1]],p0=gauss3p0)
        else:
            #print i
            if np.isnan(gausspararray[0,i-1]):
                print "Previous line didn't fit"
                gauss4par,gauss4cov=curve_fit(gaussian3,yaxis[fitrange[0]:fitrange[1]],line[fitrange[0]:fitrange[1]],p0=gauss3p0)
            else:
                nextpar=gausspararray[:,i-1]
                nextpar[1]-=8.
                nextpar[7]+=8.
                gauss4par,gauss4cov=curve_fit(gaussian3,yaxis[fitrange[0]:fitrange[1]],line[fitrange[0]:fitrange[1]],p0=nextpar)
                #gauss4par,gauss4cov=curve_fit(gaussian3,yaxis[fitrange[0]:fitrange[1]],line[fitrange[0]:fitrange[1]],p0=gausspararray[:,i-1])
        axes[1].plot(yaxis,gaussian3(yaxis,*gauss4par),'r--')
        axes[1].plot(yaxis,gaussian3(yaxis,*gauss3p0),'k--')
        
        gausspararray[:,i]=gauss4par
        #print "Gauss4par:",gauss4par
        gausserrarray[:,i]=np.sqrt(np.diag(gauss4cov))
        #axes[0].plot([x[i]],[gauss4par[1]], 'ro',color='green')
        #axes[0].plot([x[i]],[gauss4par[4]], 'ro',color='yellow')
        #axes[0].plot([x[i]],[gauss4par[7]], 'ro',color='red')
    except RuntimeError:
        print x[i], 'did not fit'
#    print gauss3par
axes[1].axvline(fitrange[0], linestyle='--', color='k')
axes[1].axvline(fitrange[1], linestyle='--', color='k')
#plt.legend()

#print 'Example parameters for x=', x[4], ':', gauss4pararray[:,4]

#print gausspararray

plt.figure()
plt.subplot(141)
plt.scatter(x,gausspararray[1,:])
plt.errorbar(x,gausspararray[1,:],yerr=gausserrarray[1,:],linestyle='None')
plt.subplot(142)
plt.scatter(x,gausspararray[4,:])
plt.errorbar(x,gausspararray[4,:],yerr=gausserrarray[4,:],linestyle='None')
plt.subplot(143)
plt.scatter(x,gausspararray[7,:])
plt.errorbar(x,gausspararray[7,:],yerr=gausserrarray[7,:],linestyle='None')


NameError: name 'trapezoiddiffr1d' is not defined

In [39]:
#----------fit linear function that describes the streak position --------------
#%matplotlib qt
def fit_streak_linear(x,y,yerr,upper_constr=None,lower_constr=None,color='red'):
    streak=y
    xstreak=x
    streakerr=yerr
    if upper_constr!=None:
        streak=streak[streak<upper_constr]
        xstreak=xstreak[streak<upper_constr]
        streakerr=streakerr[streak<upper_constr]
    if lower_constr!=None:
        streak=streak[streak>lower_constr]
        xstreak=xstreak[streak>lower_constr]
        streakerr=streakerr[streak>lower_constr]
    plt.scatter(xstreak,streak)
    plt.errorbar(xstreak,streak,streakerr,linestyle='None')
    streakpar,streakcov=curve_fit(linear,xstreak,streak,sigma=streakerr,p0=[1000.,-1.0])
    streakerr=np.sqrt(np.diag(streakcov))
    plt.plot(np.arange(2047),linear(np.arange(2047),*streakpar),color=color)
    print "Offset a=", streakpar[0], '+/-', streakerr[0]
    print "Slope b=", streakpar[1], '+/-', streakerr[1]
    ang=(np.arctan(streakpar[1])/(2.*np.pi)*360.)
    print "Angle ang=", '%.2f' % ang, '°'
    return streakpar[0], streakerr[0], streakpar[1], streakerr[1],ang
plt.imshow(np.log(im))
plt.colorbar()
print 'Lower streak:'
al,aerrl,bl,berrl,angl=fit_streak_linear(x,gausspararray[7,:],gausserrarray[7,:],color='red')
#print 'Lower streak:'
#al,aerrl,bl,berrl,angl=fit_streak_linear(x,gausspararray[10,:],gausserrarray[10,:],lower_constr=1100.)
print 'Horizontal streak:'
ah,aerrh,bh,berrh,angh=fit_streak_linear(x,gausspararray[4,:],gausserrarray[4,:],color='yellow')

print 'Upper streak:'
au,aerru,bu,berru,angu=fit_streak_linear(x,gausspararray[1,:],gausserrarray[1,:],color='green')

plt.xlim([0,600])
plt.ylim([0,600])

trapezoid_angle=[angl,angu,angh]

q0=[len(im)//2,len(im)//2]
print "q0=",q0

NameError: name 'plt' is not defined

In [40]:
q0=list(q0)
q0[0]=int(q0[0])
q0[1]=int(q0[1])

widthu=10.
widthh=5.
widthl=10.

lengthu=len(im)//2
lengthl=len(im)//2
lengthh=len(im)//2


NameError: name 'q0' is not defined

In [41]:
print angl, trapezoid_angle[0]

NameError: name 'angl' is not defined

In [42]:
#trapezoid

print 'lower:'
broadline=broad_linescan(im,trapezoid_angle[0],q0,lengthl,widthl,plot=False,mode='sum',log=True)

print 'upper:'
broadline=broad_linescan(im,trapezoid_angle[1],q0,lengthu,widthu,plot=False,mode='sum',log=True)

print 'horizontal'
broadline=broad_linescan(im,trapezoid_angle[2],q0,lengthh,widthh,plot=True,mode='sum',log=True)

NameError: name 'broad_linescan' is not defined

lower:


In [43]:
print len(diffpatterns_trapezoid), len(modes)
print modes
7 8
['sharp_filled', 'erf_filled', 'supergauss_filled', 'sharp_front', 'erf_front', 'supergauss_front', '2d_flat']


SyntaxError: invalid syntax (<ipython-input-43-63d559fd10bb>, line 3)

In [44]:
trapezoid_broadlines=[]
width=8.
#fig,axbroad=plt.subplots()
#fig2,axframe=plt.subplots()
for m in range(len(modes)):
#for m in range(2):
    print modes[m]
    #fig2.add_subplot()
    broadline=broad_linescan(diffpatterns_trapezoid[m],trapezoid_angle[0],q0,lengthu,5.,plot='section',mode='sum',log=True)
    trapezoid_broadlines.append(broadline)
    plt.title(modes[m])
    #xbroad.plot(broadline,label=modes[m])
#plt.xscale('log')
#plt.yscale('log')
#axbroad.legend(loc=3)

NameError: name 'modes' is not defined

In [45]:
for m in range(len(modes)):
    plt.plot(trapezoid_broadlines[m], label=modes[m])
plt.xscale('log')
plt.yscale('log')
plt.legend(loc=3)
plt.ylim([1e-30,1e14])

NameError: name 'modes' is not defined